<a href="https://colab.research.google.com/github/mostafa-ja/Anomaly-detection/blob/main/autoencoder7_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# we upload, in case that we cant parse logs(a lot of time and ram consumption)

# Mount Google Drive to save datasets
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#copy log file
!cp '/content/drive/MyDrive/zHDFS2/logs_train' '/content/'
!cp '/content/drive/MyDrive/zHDFS2/logs_ntest' '/content/'
!cp '/content/drive/MyDrive/zHDFS2/logs_atest' '/content/'
!cp '/content/drive/MyDrive/zHDFS2/log2index' '/content/'
!cp '/content/drive/MyDrive/zHDFS2/reduced_embeddings' '/content/'

In [3]:
import pandas as pd
import json
from sklearn.metrics.pairwise import cosine_similarity
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import os
import numpy as np


# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
device

device(type='cuda')

In [5]:
with open('/content/reduced_embeddings', 'r') as json_file:
    embeddings = json.load(json_file)

In [6]:
len(embeddings[0])


10

In [7]:
# Define the LSTM Autoencoder model with dropout
class LSTMAutoencoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, sequence_length, dropout_prob): #input_dim= embeddings_dim
        super(LSTMAutoencoder, self).__init__()
        self.encoder = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, bidirectional=True, dropout=dropout_prob)
        self.decoder = nn.LSTM(hidden_dim * 2, input_dim, num_layers, batch_first=True, dropout=dropout_prob)
        self.sequence_length = sequence_length

    def forward(self, x):
        encoded, _ = self.encoder(x)
        #print(encoded.shape)
        encoded = encoded[:,-1:,:] # output of last cell
        #print(encoded.shape)
        input_decode = torch.tile(encoded, (1, self.sequence_length, 1))
        decoded, _ = self.decoder(input_decode)
        return decoded

In [8]:
sequence_length = 5
input_size = 10 #embedding vector dimension
hidden_dim = 32
num_layers = 2
dropout_prob = 0.2

model = LSTMAutoencoder(input_size, hidden_dim, 2, sequence_length, dropout_prob).to(device)
model

LSTMAutoencoder(
  (encoder): LSTM(10, 32, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (decoder): LSTM(64, 10, num_layers=2, batch_first=True, dropout=0.2)
)

In [9]:
data = torch.randn(1,sequence_length, input_size) #batch_size=1
model(data.to(device)).shape

torch.Size([1, 5, 10])

In [10]:
# Calculate the number of parameters
num_params = sum(p.numel() for p in model.parameters())
print(f"Number of Parameters: {num_params}")

Number of Parameters: 40272


In [11]:

name = 'logs_train'
window_size = sequence_length
num_sessions = 0
inputs = []
#outputs = []

with open('/content/' + name, 'r') as f:
        for row in f:
            num_sessions += 1
            line = [ embeddings[int(i)] for i in row.strip().split()]
            for i in range(len(line) - window_size):
                inputs.append(line[i:i + window_size])
                #outputs.append(line[i + window_size])

print('Number of sessions({}): {}'.format(name, num_sessions))
print('Number of seqs({}): {}'.format(name, len(inputs)))
#dataset = TensorDataset(torch.tensor(inputs, dtype=torch.float), torch.tensor(outputs))
dataset = TensorDataset(torch.tensor(inputs, dtype=torch.float))

Number of sessions(logs_train): 446578
Number of seqs(logs_train): 6936890


In [12]:
dataloader = DataLoader(dataset, batch_size=256, shuffle=True)
len(dataloader) # regarding to batch size

27098

In [13]:
for step, (seq) in enumerate(dataloader):
  print(seq[0].shape)
  break

torch.Size([256, 5, 10])


In [14]:
def adjust_learning_rate(optimizer, epoch, lr_step=(10,15,20,25), lr_decay_ratio=0.2):
    """Adjust the learning rate based on the epoch number."""
    if epoch == 0:
        optimizer.param_groups[0]['lr'] /= 8
    elif epoch in [1, 2, 3]:  # in step five , we finish warm up ,and start normal learning rate
        optimizer.param_groups[0]['lr'] *= 2
    if epoch in lr_step: # in these steps , we are geting close to optimal point so we need to have shorter step
        optimizer.param_groups[0]['lr'] *= lr_decay_ratio
    return optimizer

In [15]:
num_epochs = 45
learning_rate = 0.001

# Loss and optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999))
criterion = nn.MSELoss()

In [ ]:
# Train the model
model.train()
start_time = time.time()
total_step = len(dataloader)
for epoch in range(num_epochs):  # Loop over the dataset multiple times
    optimizer = adjust_learning_rate(optimizer, epoch)
    train_loss = 0
    for step, (seq) in enumerate(dataloader):
        # Forward pass
        seq = seq[0].clone().detach().view(-1, window_size, input_size).to(device)
        output = model(seq)
        loss = criterion(output, seq.to(device))

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
    print('Epoch [{}/{}], train_loss: {:.4f}'.format(epoch + 1, num_epochs, train_loss / total_step))
elapsed_time = time.time() - start_time
print('elapsed_time: {:.3f}s'.format(elapsed_time))
print('Finished Training')


Epoch [1/45], train_loss: 0.0063
Epoch [2/45], train_loss: 0.0021
Epoch [3/45], train_loss: 0.0015
Epoch [4/45], train_loss: 0.0011
Epoch [5/45], train_loss: 0.0010
Epoch [6/45], train_loss: 0.0009
Epoch [7/45], train_loss: 0.0009
Epoch [8/45], train_loss: 0.0008
Epoch [9/45], train_loss: 0.0008
Epoch [10/45], train_loss: 0.0008
Epoch [11/45], train_loss: 0.0008
Epoch [12/45], train_loss: 0.0008
Epoch [13/45], train_loss: 0.0008
Epoch [14/45], train_loss: 0.0007
Epoch [15/45], train_loss: 0.0007
Epoch [16/45], train_loss: 0.0007
Epoch [17/45], train_loss: 0.0007


In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/Autoencoder3_parameters.pth')

# **EVALUATION**


in cpu
```
my_model = model.load_state_dict(torch.load('model_parameters.pth', map_location=torch.device(device)))

# Put the model in evaluation mode if necessary
model.eval()
```

in cuda


```
model.load_state_dict(torch.load('model_parameters.pth'))

# Put the model in evaluation mode if necessary
model.eval()
```



In [ ]:
len(embeddings)

In [ ]:
embeddings.append([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

In [ ]:
embeddings[len(embeddings)-1]

In [ ]:
def generate(name):
    window_size = sequence_length
    hdfs = {} #store the unique sequences and their counts.
    length = 0
    with open('/content/' + name, 'r') as f:
        for row in f:
            line = [int(i) for i in row.strip().split()]
            line = line + [len(embeddings)-1] * (window_size + 1 - len(line)) #if the length of the line is less than windows size, it covers by 30 a template with zeros vector
            hdfs[tuple(line)] = hdfs.get(tuple(line), 0) + 1   #If the tuple is not present in the dictionary, hdfs.get(tuple(ln), 0) returns 0, and the code initializes the count to 1.
            length += 1
            # hdfs.append(tuple(line))
    print('Number of sessions({}): {}'.format(name, len(hdfs)))
    return hdfs, length

In [ ]:
test_normal_loader, test_normal_length = generate('logs_ntest')
test_abnormal_loader, test_abnormal_length = generate('logs_atest')

In [ ]:
def evaluation(threshold):
  # Test the model
  model.eval()

  TP = 0
  FP = 0

  start_time = time.time()
  with torch.no_grad():
      for line in test_normal_loader.keys():
          for i in range(len(line) - window_size):
              session = line[i:i + window_size]
              seq = [embeddings[temp] for temp in session]
              seq = torch.tensor(seq, dtype=torch.float).view(-1, window_size, input_size).to(device)
              output = model(seq)

              loss = criterion(output, seq)

              if (loss.cpu().detach().numpy()>threshold):
                FP += test_normal_loader[line] # numbers of that set we have
                break
  with torch.no_grad():
      for line in test_abnormal_loader.keys():
          for i in range(len(line) - window_size):
              session = line[i:i + window_size]
              seq = [embeddings[temp] for temp in session]
              seq = torch.tensor(seq, dtype=torch.float).view(-1, window_size, input_size).to(device)
              output = model(seq)

              loss = criterion(output, seq)

              if (loss.cpu().detach().numpy()>threshold):
                TP += test_abnormal_loader[line]
                break
  elapsed_time = time.time() - start_time
  print('elapsed_time: {:.3f}s'.format(elapsed_time))
  # Compute precision, recall and F1-measure
  FN = test_abnormal_length - TP
  P = 100 * TP / (TP + FP)
  R = 100 * TP / (TP + FN)
  F1 = 2 * P * R / (P + R)
  print('false positive (FP): {}, false negative (FN): {}, Precision: {:.3f}%, Recall: {:.3f}%, F1-measure: {:.3f}%'.format(FP, FN, P, R, F1))
  print('Finished Predicting')

In [ ]:
threshold = [0.007,0.008,0.009,0.01]
for i in threshold:
  print('-------------------------------------------------------------------------')
  print('threshold = ', i)
  evaluation(i)

In [ ]:
threshold = [0.00726,0.00727,0.00728,0.00729]
for i in threshold:
  print('-------------------------------------------------------------------------')
  print('threshold = ',i)
  evaluation(i)